In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

Download dữ liệu chữ số viết tay MNIST

In [3]:
# Chuẩn bị dữ liệu
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Chuyển đổi sang định dạng float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Chuẩn hóa ảnh từ from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test, y_train, y_test = torch.from_numpy(x_train), torch.from_numpy(x_test), torch.from_numpy(y_train).type(torch.LongTensor), torch.from_numpy(y_test).type(torch.LongTensor)

2025-10-18 09:37:11.333785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760780231.521768      77 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760780231.572493      77 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [19]:
print(y_train.shape)

torch.Size([60000])


In [43]:
batch_size = 16
trainloader = []
for (i,j) in zip(x_train, y_train):
    trainloader.append([i,j])
trainloader = torch.utils.data.DataLoader(trainloader, shuffle=True, batch_size=batch_size)

testloader = []
for (i,j) in zip(x_test, y_test):
    testloader.append([i,j])
testloader = torch.utils.data.DataLoader(testloader, shuffle=True, batch_size=batch_size)

In [26]:
num_features = 784
n_hidden_1 = 512
n_hidden_2 = 128
n_hidden_3 = 32
num_classes = 10

Sử dụng các tham số ở trên để xây dựng mô hình

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                                nn.Linear(num_features, n_hidden_1),
                                nn.ReLU(),
                                nn.Linear(n_hidden_1, n_hidden_2),
                                nn.ReLU(),
                                nn.Linear(n_hidden_2, n_hidden_3),
                                nn.ReLU(),
                                nn.Linear(n_hidden_3, num_classes)
        )

    def forward(self, x):
        return self.net(x)

net = Net()

In [28]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [34]:
for epoch in range(10):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # load input và labels
        input, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        input_flatten = input.reshape((len(input), -1))

        #print (input_flatten.shape)
        # forward + backward + optimize
        outputs = net(input_flatten)
        #print(labels.shape)
        #print(input.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.001
[2,  2000] loss: 0.000
[3,  2000] loss: 0.000
[4,  2000] loss: 0.000
[5,  2000] loss: 0.000
[6,  2000] loss: 0.000
[7,  2000] loss: 0.000
[8,  2000] loss: 0.000
[9,  2000] loss: 0.000
[10,  2000] loss: 0.000
Finished Training


In [47]:
correct = 0
total = 0
# do đang thực hiện việc dự đoán nên ko cần tính đạo hàm
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        #print(inputs.shape)
        input_flatten = inputs.reshape(len(inputs), -1)
        #print(input_flatten.shape)
        # chạy hàm dự đoán
        outputs = net(input_flatten)
        # the class với giá trị xác suất cao nhất là đâu ra dự đoán
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


## Save and load model<br>
Trình bày 1 trong các các lưu model và load model trong PyTorch

1.   Lưu model<br>
```
torch.save(model.state_dict(), PATH)
```
trong đó PATH là đường dẫn tự định nghĩa
2.   Load model <br>

*   Trước tiên phải định nghĩa model trước. Model được định nghĩa phải giống hệt với model đã được lưu lại. Như ví dụ trong bài này, thì sẽ thực hiện như sau: 
```
model = Net()
```
*   Load trọng số đã được học vào mô hình<br>
```
model.load_state_dict(torch.load(PATH))
# vô hiệu hóa các layer như Dropout hay BatchNorm 
model.eval()
```
3. Có thể tham khảo thêm các phương pháp lưu và load model tại: https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html








In [53]:
# SAVE MODEL PARAM
import torch
import torchvision.models as models

torch.save(net.state_dict(), 'model_weights.pth')

In [54]:
net = Net()                     # tạo lại cùng kiến trúc
net.load_state_dict(torch.load('model_weights.pth'))
net.eval()                      # chuyển sang chế độ đánh giá (bỏ dropout, batchnorm update)


Net(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [55]:
state_dict = torch.load('model_weights.pth')
for name, param in state_dict.items():
    print(name, "\t", param.shape)


net.0.weight 	 torch.Size([512, 784])
net.0.bias 	 torch.Size([512])
net.2.weight 	 torch.Size([128, 512])
net.2.bias 	 torch.Size([128])
net.4.weight 	 torch.Size([32, 128])
net.4.bias 	 torch.Size([32])
net.6.weight 	 torch.Size([10, 32])
net.6.bias 	 torch.Size([10])


In [57]:
print(state_dict['net.0.weight'])


tensor([[ 0.0292,  0.0328,  0.0089,  ...,  0.0335,  0.0132,  0.0034],
        [ 0.0312, -0.0050,  0.0105,  ..., -0.0176, -0.0019,  0.0340],
        [ 0.0100,  0.0305, -0.0286,  ...,  0.0175,  0.0234, -0.0344],
        ...,
        [ 0.0100, -0.0212,  0.0329,  ..., -0.0322, -0.0221,  0.0110],
        [ 0.0247, -0.0315, -0.0162,  ...,  0.0035, -0.0262,  0.0063],
        [ 0.0165, -0.0332, -0.0343,  ...,  0.0119, -0.0112,  0.0330]])
